In [30]:
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
import numpy as np
import requests

In [31]:
def SMA(data, width):
    i = 0
    moving_averages = []
    while i < len(data) - width + 1:
        window = data[i : i + width]
        window_average = round(np.sum(window) / width, 2)
        moving_averages.append(window_average)
        i += 1
    return moving_averages

In [32]:
start = 1679950001 - 2*3600
start = 1679934272 - 2*3600
end = (datetime.today() + timedelta(hours=0)).timestamp()
from bs4 import BeautifulSoup
prometheus_info = response = requests.get("http://wally193.cit.tu-berlin.de:9091/status")
prometheus_info_soup= BeautifulSoup(prometheus_info.text, 'html.parser')

start = prometheus_info_soup.findAll('td')[0].text.split('.')[0]
datetime_object = datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
start = datetime_object.timestamp()

query = "node_memory_MemTotal_bytes - node_memory_MemAvailable_bytes"

url = f'http://wally193.cit.tu-berlin.de:9091/api/v1/query_range?query={query}&end={end}&step=60&start={start}'
print(url)
response = requests.get(url, headers={'Accept': 'application/json'})
response = response.json()['data']['result']
metrics = dict()
for instance in response:
    node = instance['metric']['instance']
    values = instance['values']
    metrics[node]=values
    print(len(values))

plt.figure().set_figwidth(15)
for instance, values in metrics.items():
    data = [int(point[1]) for point in values]
    timestamp =  [int(point[0]) for point in values]
    timestamp = np.array(timestamp)
    timestamp = np.subtract(timestamp, timestamp.min())
    timestamp = np.divide(timestamp, 3600)
    data = np.divide(np.array(data), 1073741824)
    plt.plot(timestamp[:-9], SMA(data, 10), label=instance)
plt.legend()
plt.show()

TypeError: 'module' object is not callable